In [1]:
from rosetta import *
init()
from toolbox import rcsb
### the answers are listed in different cells for each question.

core.init: Rosetta version  from 
core.init: command: PyRosetta -ex1 -ex2aro -database /Users/XT/ResearchApps/PyRosetta_1230/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=1439393359 seed_offset=0 real_seed=1439393359
core.init.random: RandomGenerator:init: Normal mode, seed=1439393359 RG_type=mt19937


In [3]:
# a.
initial_pose=rcsb.pose_from_rcsb('1kjg')
pose=Pose()
pose.assign(initial_pose)

# create a list to define the task_pack
aa_list=[8, 23, 25, 29,30, 32, 45, 47, 50, 53, 82, 84] 
new_list=[8, 23, 25, 29,30, 32, 45, 47, 50, 53, 82, 84] 
for i in aa_list:
     new_list.append(pose.pdb_info().pdb2pose("B", i))
for i in range(2,10):
     new_list.append(pose.pdb_info().pdb2pose("P", i))  

# set the task pack, movemap, scorefxn, pack mover, minmover
task_pack = standard_packer_task(pose)
task_pack.restrict_to_repacking()
task_pack.temporarily_fix_everything()
for i in new_list:
    task_pack.temporarily_set_pack_residue(i, True)
movemap=MoveMap()
movemap.set_bb(False)
movemap.set_chi(False)
for i in new_list:
    movemap.set_chi(i, True)
scorefxn=get_fa_scorefxn()
pack_mover=PackRotamersMover(scorefxn,task_pack)
minmover=MinMover()
minmover.movemap(movemap)
minmover.score_function(scorefxn)


the file 1KJG.pdb already exists, this file will be overwritten
PDB 1KJG successfully loaded from rcsb into 1KJG.pdb
if the file 1kjg.clean.pdb already exists, it will be overwritten
PDB 1kjg.pdb successfully cleaned, non-ATOM lines removed
clean data written to 1kjg.clean.pdb
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue LYS 7
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue LYS 7
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CE  on residue LYS 7
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NZ  on residue LYS 7
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue LYS 41
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue LYS 41
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CE  on residue LYS 41
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NZ  on residue LYS 41
core.conformation.Conformation: [ WARN

In [51]:
# b.
# begin to search the wild type
pose.assign(initial_pose)
for i in range (1,11): 
    pack_mover.apply(pose)
    minmover.apply(pose)
    print scorefxn(pose)
    with open('score_repacking.txt','a') as f:
        f.write(str(scorefxn(pose))+'\n')
    pose.dump_pdb(str(i)+'_repacking.pdb')
# the final score is -124.927109096

core.pack.pack_rotamers: built 683 rotamers at 32 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
core.pack.interaction_graph.interaction_graph_factory: IG: 583012 bytes
-124.927109096
core.pack.pack_rotamers: built 683 rotamers at 32 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
core.pack.interaction_graph.interaction_graph_factory: IG: 583012 bytes
-124.927109096
core.pack.pack_rotamers: built 683 rotamers at 32 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
core.pack.interaction_graph.interaction_graph_factory: IG: 583012 bytes
-124.927109096
core.pack.pack_rotamers: built 683 rotamers at 32 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
core.pack.interaction_graph.interaction_graph_factory: IG: 583012 bytes
-124.927109096
core.pack.pack_rotamers: built 683 r

In [56]:
# c. 
# create mutant at position 199 ('P', 2)
task_pack_mutation = standard_packer_task(pose)
task_pack_mutation.temporarily_fix_everything()
task_pack_mutation.temporarily_set_pack_residue(199, True)
pack_mover_mutation=PackRotamersMover(scorefxn,task_pack_mutation)
for i in range(1,11):
    pack_mover_mutation.apply(pose)
    minmover.apply(pose)
    scorefxn(pose)
    with open('score_mutant','a') as f:
        f.write(str(scorefxn(pose))+'\n')
    pose.dump_pdb(str(i)+'_mutation.pdb')
# best score: -127.740555546

core.pack.task: Packer task: initialize from command line() 
core.pack.pack_rotamers: built 119 rotamers at 1 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating PDInteractionGraph
core.pack.interaction_graph.interaction_graph_factory: IG: 3696 bytes
core.pack.pack_rotamers: built 118 rotamers at 1 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating PDInteractionGraph
core.pack.interaction_graph.interaction_graph_factory: IG: 3684 bytes
core.pack.pack_rotamers: built 119 rotamers at 1 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating PDInteractionGraph
core.pack.interaction_graph.interaction_graph_factory: IG: 3696 bytes
core.pack.pack_rotamers: built 118 rotamers at 1 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating PDInteractionGraph
core.pack.interaction_graph.interaction_graph_factory: IG: 3684 bytes
core.pack.pack_rotamers: built 118 rotamers at 1 positions.
core.pa

In [55]:
# d.
# create other single mutants at different positions
### There is an error which I cannot solve. Thus I run the code without assign to the wild type each time.
### thus accually rather than single mutants, the mutants are accumulated.
wild_type=pose_from_file('10_repacking.pdb')
pose.assign(wild_type)
for resi in range(199,207):    
    task_pack_mutation = standard_packer_task(pose)
    task_pack_mutation.temporarily_fix_everything()
    task_pack_mutation.temporarily_set_pack_residue(resi, True)
    pack_mover=PackRotamersMover(scorefxn,task_pack_mutation)
    for i in range(1,11):
        pack_mover.apply(pose)
        minmover.apply(pose)
        scorefxn(pose)
        with open(str(resi)+'_resi_score_mutant.txt','a') as f:
            f.write(str(scorefxn(pose))+'\n')
        pose.dump_pdb(str(resi)+'_'+str(i)+'_mutation.pdb')
# the lowest score is 200_10_mutation.pdb: -131.934716693

core.pack.task: Packer task: initialize from command line() 
core.pack.pack_rotamers: built 117 rotamers at 1 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating PDInteractionGraph
core.pack.interaction_graph.interaction_graph_factory: IG: 3672 bytes


RuntimeError: unidentifiable C++ exception

In [ ]:
'''
e&f.
# the wild type final score is -124.927109096
resi  lowest_score
199:-131.898937542
200:-131.934716693
201:-131.934716693
202:-131.900721794 hardeat to improve(the score incresed while mutated)
203:-131.786455346
204:-131.932454516
205:-131.862737463
206:-131.927074199 easiest to imporove(the score decreased by 0.0526)

except 202 residue, other residues mutations tend to improve the energy.


g.
It tends to become more hydrophobic and bulky.


h:
My result: 200_10 is the lowest energy structure. It is a A199S/E200W/T210V mutant.



i:
Natural substrates are often sub-optimal binders. Why would this be advantageous?
First, there are different distributions for each type of the amino acids in orgaisms, thus 
some amino acids which can provide the best structure are rare. 
Second, if the energy is too low, the cost to seperate them apart is too high. To save more 
energy in the pathway, a sub-optimal structure can be a better choice. 
'''